Author: Tyler Chase

Date: 2017/05/18

# Milestone Model Subreddit Classification

This code uses a classification model to classify an image to one of 20 subreddits. For the sake of the milestone we implement an Alex Net model and analyze the results. 

## Wrapper function for building graph

This function reduces the text needed for running the @property wrapper making the code more readable. 

In [ ]:
# reduces the text needed for running @property making code more readable
def lazy_property(function):
    # Attribute used to test if code chunk has been run or not
    attribute = '_lazy_' + function.__name__
    # run wrapper function when wrapper returned below
    @property
    # Keeps original function attributes such as function.__name__ 
    # Otherwise it would be replaced with the wrapper attributes
    @functools.wraps(function)
    def wrapper(self):
        # If doesn't have (attribute) then code chunk hasn't been run
        if not hasattr(self, attribute):
            # Run code chunk and store it in (attribute) of class
            setattr(self, attribute, function(self))     
        # return the value of the number stored in (attribute)
        return getattr(self, attribute)
    return wrapper

## Load Datasets

In [ ]:
import tensorflow as tf
import numpy as np
import math
import timeit
import random
import pickle
import matplotlib.pyplot as plt
import itertools
import functools
import time
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [ ]:
# Load data and split into training validation and testing sets

# Function for permuting and splitting data into training, developement, and test
def import_dataset(address, file_names, train_percent = 80, dev_percent = 10):
    SEED = 455
    random.seed(SEED)
    # Read csv file and create a list of tuples
    images = np.load(address+file_names['images'])
    images = images.astype(float)
    with open(address + file_names['subs'], 'rb') as file_2:
        subs = pickle.load(file_2)
        subs = np.array(subs)
    with open(address + file_names['dict'], 'rb') as file_3:
        dictionary = pickle.load(file_3)
    # Mix data and split into tran, dev, and test sets
    N,W,H,C = np.shape(images)
    indices = np.arange(N)
    random.shuffle(indices)
    images = images[indices]
    subs = subs[indices]
    train_end = int(train_percent*N/100)
    dev_end = train_end + int(dev_percent*N/100)
    X_train = images[:train_end]
    y_train = subs[:train_end]
    X_val = images[train_end:dev_end]
    y_val = subs[train_end:dev_end]
    X_test = images[dev_end:]
    y_test = subs[dev_end:]
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    return X_train, y_train, X_val, y_val, X_test, y_test, dictionary

# Form training, developement, and testing data sets
address = r'/Users/tylerchase/Documents/Stanford_Classes/CS231n_CNN_for_Visual_Recognition/final_project/milestoneData/Archive//'
file_names = {}
file_names['images'] = 'final_output_data.npy'
file_names['subs'] = 'final_output_labels'
file_names['dict'] = 'final_output_subredditIndex'
X_train, y_train, X_val, y_val, X_test, y_test, dictionary = import_dataset(address, file_names)

# Print the sizes as a sanity check
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)


## Define AlexNet model 

* 11x11 convolutional layer with 96 filters and a stride of 4
* ReLU activation
* 3x3 max pooling with a stride of 2
* batch normalization


* 5x5 convolutional layer with 256 filters and a stride of 1
* ReLU activation
* 3x3 max pooling with a stride of 2
* batch normalization


* 3x3 convolutional layer with 384 filters and a stride of 1
* ReLU activation
* 3x3 convolutional layer with 384 filters and a stride of 1
* ReLU activation 
* 3x3 convolutional layer with 256 filters and a stride of 1
* ReLU activation
* 3x3 max pooling with a stride of 2


* affine layer from 4096 to 1792
* ReLU activation
* affine layer from 1792 to 1792
* ReLU activation
* affine layer from 1792 to 20

In [ ]:
# Configuration file storing important values
class Config:
    def __init__(self, sample_size, class_size, image_width, image_height, image_depth):
        self.sample_size = sample_size
        self.class_size = class_size
        self.image_width = image_width
        self.image_height = image_height
        self.image_depth = image_depth

In [ ]:
class AlexNet:
    
    def __init__(self, config, learning_rate=0.003):
        self.config = config
        self.data, self.target, self.is_training = self._initialize_placeholders()
        self.learning_rate = learning_rate
        self.prediction
        self.cost
        self.optimize
        self.accuracy
        
    def _initialize_placeholders(self):
        data = tf.placeholder(tf.float32, [None, self.config.image_height, 
                                         self.config.image_width, self.config.image_depth])
        target = tf.placeholder(tf.int64, [None])
        is_training = tf.placeholder(tf.bool)
        return(data, target, is_training)
        
    @lazy_property
    def prediction(self):
        # define our graph (e.g. AlexNet)
        
        a1 = tf.layers.conv2d(self.data, filters=96, kernel_size=(11,11), strides=(4,4), padding='SAME') #Out N/4 x N/4
        h1 = tf.nn.relu(a1)
        mp1 = tf.layers.max_pooling2d(h1, pool_size=(3,3), strides=(2,2), padding='SAME')    #Out N/2 x N/2
        bn1 = tf.layers.batch_normalization(mp1, training=self.is_training)
        
        a2 = tf.layers.conv2d(bn1, filters=256, kernel_size=(5,5), strides=(1,1), padding='SAME')     #Out N x N
        h2 = tf.nn.relu(a2)
        mp2 = tf.layers.max_pooling2d(h2, pool_size=(3,3), strides=(2,2), padding='SAME')    #Out N/2 x N/2
        bn2 = tf.layers.batch_normalization(mp2, training=self.is_training)              
    
        a3 = tf.layers.conv2d(bn2, filters=384, kernel_size=(3,3), strides=(1,1), padding='SAME')     #Out N x N
        h3 = tf.nn.relu(a3)
        a4 = tf.layers.conv2d(h3, filters=384, kernel_size=(3,3), strides=(1,1), padding='SAME')      #Out N x N
        h4 = tf.nn.relu(a4)
        a5 = tf.layers.conv2d(h4, filters=256, kernel_size=(3,3), strides=(1,1), padding='SAME')      #Out N x N
        h5 = tf.nn.relu(a5)
        mp3 = tf.layers.max_pooling2d(h5, pool_size=(3,3), strides=(2,2), padding='SAME')   #Out N/2 x N/2
    
        mp_flat = tf.reshape(mp3,[-1,4096])
        aff1 = tf.layers.dense(mp_flat, 1792)
        h6 = tf.nn.relu(aff1)
        aff2 = tf.layers.dense(h6, 1792)
        h7 = tf.nn.relu(aff2)
        y_out = tf.layers.dense(h7, self.config.class_size)
    
        return y_out        
    
    @lazy_property
    def cost(self):
        target_vec = tf.one_hot(self.target, self.config.class_size)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=target_vec, logits=self.prediction)
        #cross_entropy = tf.losses.hinge_loss(labels=target_vec, logits=self.prediction)
        cross_entropy_sum = tf.reduce_sum(cross_entropy)
        return cross_entropy_sum
    
    @lazy_property
    def optimize(self):
        opt = tf.train.AdamOptimizer(self.learning_rate)
        #opt = tf.train.GradientDescentOptimizer(self.learning_rate)

        train_step = opt.minimize(self.cost)
        
        # batch normalization in tensorflow requires this extra dependency
        # this is required to update the moving mean and moving variance variables
        extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(extra_update_ops):
            train_step = opt.minimize(self.cost)
        
        return(train_step)
    
    @lazy_property
    def accuracy(self):
        incorrect = tf.equal(tf.argmax(self.prediction, axis = 1), self.target)
        return tf.reduce_mean( tf.cast(incorrect, tf.float32) )

## Test the Size of Your Output as Sanity Check

Input random numbers into X for feed dictionary and check the size of the output makes sense. 

In [ ]:
# Now we're going to feed a random batch into the model 
# and make sure the output is the right size
config = Config(sample_size=100, class_size=20, image_height=128, image_width=128, image_depth=3)
x = np.random.randn(config.sample_size, config.image_height, config.image_width, config.image_depth)
y = np.random.randn(config.sample_size)
with tf.Session() as sess:
    with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0"

        model = AlexNet(config, learning_rate=0.003)
        
        tf.global_variables_initializer().run()
        variables = [model.prediction, model.cost, model.accuracy]
        ans_rnd, cost_rnd, acc_rnd = sess.run(variables,feed_dict={model.data:x, model.target:y, model.is_training:True})
        ans, cost, acc = sess.run(variables,feed_dict={model.data:X_val, model.target:y_val, model.is_training:True})
       
        print('random data')
        print('prediction shape: ',ans_rnd.shape)
        print(np.array_equal(ans_rnd.shape, np.array([100, 20])))
        print('cost: ', cost_rnd)
        print('accuracy: ', acc_rnd)
        print('\nvalidation data')
        print('prediction shape: ',ans.shape)
        print('cost: ', cost)
        print('accuracy: ', acc)

## Train the Model

In [ ]:
# Data class for run
class Data:
    def __init__(self, X_train, Y_train, X_val, Y_val, X_test, Y_test):
        self.X_train = X_train
        self.Y_train = Y_train
        self.X_val = X_val
        self.Y_val = Y_val
        self.X_test = X_test
        self.Y_test = Y_test

In [ ]:
# Function for running model
class Run:
    
    def __init__(self, session, config, data, model, epoch_size=15, minibatch_size = 100, 
                 learning_rate = 0.003, save_flag = False, print_every = 100, saver_address='',
                 loader_address=''):
        self.session = session 
        self.config = config
        self.data = data
        self.model = model
        self.epoch_size = epoch_size
        self.minibatch_size = minibatch_size
        self.learning_rate = learning_rate
        self.save_flag = save_flag
        self.print_every = print_every
        self.saver_address = saver_address
        self.loader_address = loader_address
        self._train_loss_hist = []
        self._val_loss_hist = []
        self._train_acc_hist = []
        self._val_acc_hist = []
        
    def train(self):
        
        # Save model parameters
        if self.save_flag == True:    
            saver = tf.train.Saver()
            
        self.session.run( tf.global_variables_initializer() )
        
        # Loop over epochs
        for epoch in range(self.epoch_size):
            startTime_epoch = time.clock()
            startTime_batch = time.clock()
            
            # Loop over minibatches
            for j,i in enumerate(np.arange(0, self.config.sample_size, self.minibatch_size)):
                batch_x = self.data.X_train[i:i+self.minibatch_size,:,:,:]
                batch_y = self.data.Y_train[i:i+self.minibatch_size]
                self.session.run(self.model.optimize, 
                                 {self.model.data:batch_x, self.model.target:batch_y, self.model.is_training:True})
                
                # print run time, current batch, and current epoch
                if not j%self.print_every:
                    batch_time = time.clock() - startTime_batch
                    startTime_batch = time.clock()
                    print("Batch {:d}/{:d} of epoch {:d} finished in {:f} seconds".format(j+1,  \
                        int(self.config.sample_size/self.minibatch_size), (epoch+1), batch_time))
             
            # Print current output, return losses, and return accuracies
            epoch_time = time.clock() - startTime_epoch
            print("Epoch {:d} finished in {:f} seconds".format( (epoch + 1), epoch_time))
            variables = [self.model.cost, self.model.accuracy]
            loss_train, acc_train = self.session.run(variables, \
                {self.model.data:self.data.X_train, self.model.target:self.data.Y_train, self.model.is_training:False})
            loss_val, acc_val = self.session.run(variables, \
                {self.model.data:self.data.X_val, self.model.target:self.data.Y_val, self.model.is_training:False})
            print('Epoch:{:2d}, Training Accuracy {:3.1f}%, Test Accuracy:{:3.1f}%'.format( \
                (epoch + 1), (100*acc_train), (100*acc_test)))
            
            # Append losses and accuracies to list
            self._train_loss_hist.append(loss_train)
            self._val_loss_hist.append(loss_val)
            self._train_acc_hist.append(acc_train)
            self._val_acc_hist.append(acc_val)
            
            # Save model
            if self.save_flag == True: 
                # Save trained model to data folder
                saver.save(sess, self.saver_address + 'classification_model')    
                
    def load(self):
        saver = tf.train.Saver()
        saver.restore(self.session, self.loader_address)   
            
    def return_loss_acc(self):
        return self._train_loss_hist, self._val_loss_hist, self._train_acc_hist, self._val_acc_hist
            


In [ ]:
# Generate Config instance
config = Config(sample_size=np.shape(X_val)[0], class_size=20, image_height=128, image_width=128, image_depth=3)

# Generate Data instance
data = Data(X_train, y_train, X_val, y_val, X_test, y_test)

In [ ]:
# Create model instance
tf.reset_default_graph()
model = AlexNet(config, learning_rate = 0.003)
# Create run instance
session = tf.Session()
run = Run(session, config, data, model, epoch_size=1, learning_rate=0.003, print_every = 1, minibatch_size=10)
run.train()

## Run the Model on the Training and Validation Data

In [ ]:
print('Training')
run_model(sess,y_out,mean_loss,X_train,y_train,1,64)
print('Validation')
run_model(sess,y_out,mean_loss,X_val,y_val,1,64)

## Output Predictions for Validation 

In [ ]:
y_val_pred = sess.run(y_out, {X: X_val, y: y_val, is_training: 0})

y_val_pred = np.argmax(y_val_pred, axis = 1)

## Plot Confusion Matrix for Subreddit Classification

In [ ]:
# Code to plot the confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues,
                          save_address = ''):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(11,11))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i, j],2),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    #plt.savefig(save_address + 'confusion_mat.png')

classes = [""] * len(dictionary)
for sub, ind in dictionary.items():
    classes[ind] = sub

conf = confusion_matrix(y_val, y_val_pred)
plot_confusion_matrix(conf, classes=classes, normalize = True)
